In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from yieldcurves.parametric import NelsonSiegelSvensson
from yieldcurves.tools import ascii_plot, lin, plot
from yieldcurves import *
from yieldcurves.analytics import *

In [2]:
# c = NelsonSiegelSvensson.download(-1)
c = NelsonSiegelSvensson(beta0=1.9137, beta1=1.97448, beta2=-28.16468, beta3=27.22348, tau1=4.6538, tau2=5.29223)
setattr(c, 'repr_repr', r'curve()')

z = ZeroRate(c, cash_frequency=12)
df = Df(z)
c

curve()

In [ ]:
c = Const(0.01)
z = ZeroRate(c, cash_frequency=2)
df = Df(z)

In [3]:
T = 3
dates = list(x / z.cash_frequency for x in range(T * z.cash_frequency)) + [T]
sum(df(x) / z.cash_frequency for x in dates[1:])

2.8655082156929907

In [ ]:
swap_annuity(z, dates)

In [ ]:
swap_annuity(z, T, cash_frequency=z.cash_frequency)

In [ ]:
annuity = Annuity(z)
annuity, annuity(T)

In [ ]:
swap_par_rate(z, T, cash_frequency=z.cash_frequency)

In [ ]:
par = Par(z)
par, par(T), par(T, 15), z.par(T), z.par(T, 15), z.par

In [ ]:
Cash(z), Cash(z)(T), z.cash(T), z.cash

In [5]:
Df(z).cash(5), z.cash(5)

(0.02272842232334682, 0.02272842232334682)

In [ ]:
#_ = plot(lin(0.25, 30, 0.25), z, Par(z), Cash(z), Cash(z, cash_frequency=1),legend=True)
_ = plot(lin(0.25, 25, 0.25), Cash(z), Cash(z, cash_frequency=1), Par(z), Par(z, cash_frequency=1), legend=True)

In [ ]:
zcurve = ZeroRate(0.01)
df = Df(zcurve)
zcurve, df, df(5, 10), zcurve.df(5, 10), df.zero(5, 10), zcurve(5, 10)

In [ ]:
zcurve = ZeroRate(0.01)
fwd = CashRate(ZeroRate(0.01), frequency=4)
swp = Par(zcurve, forward_curve=fwd)
swp

In [ ]:
df = Df(zcurve)
zcurve.cash(10), df.cash(10), Cash(df)(10)

In [ ]:
z = ZeroRate(c, frequency=1)

In [ ]:
c = NelsonSiegelSvensson.download(-1)
setattr(c, 'repr_repr', r'$i(t)$')
d = NelsonSiegelSvensson.download(-1)
d.short_rate = True
setattr(d, 'repr_repr', r'$r(t)$')

In [ ]:
ascii_plot(lin(0, 30, 0.025), c, d)

In [ ]:
_ = plot(lin(0, 30, 0.025), c, d, legend=True)

In [ ]:
f = NelsonSiegelSvensson.download('2024-04-16')
f_1m = Cash(ZeroRate(f, frequency=12))
f_3m = Cash(ZeroRate(f, frequency=4))
f_6m = Cash(ZeroRate(f, frequency=2))
f12m = Cash(ZeroRate(f, frequency=1))
s = NelsonSiegelSvensson.download('2024-04-16')
s.short_rate = True
setattr(f, 'repr_repr', r'$i$')
setattr(f_1m, 'repr_repr', r'$f_{\tt 1m}$')
setattr(f_3m, 'repr_repr', r'$f_{\tt 3m}$')
setattr(f_6m, 'repr_repr', r'$f_{\tt 6m}$')
setattr(f12m, 'repr_repr', r'$f_{\tt 12m}$')
setattr(s, 'repr_repr', r'$r$')

In [ ]:
ax = plot(lin(0, 10, 0.025), f, f_1m, f_3m, f_6m, f12m, s, legend=True, show=False)
ax.set_ylim([0.02, 0.03])